In [15]:
!pip install Sastrawi pandas

In [16]:
import pandas as pd
import re
import pickle
import nltk
import numpy as np
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
from tqdm import tqdm
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [17]:
file_path = '/content/data_100.csv'
data = pd.read_csv(file_path)
data

,judul,tanggal,isi,kategori
0,FOTO: Melihat Kehancuran Gaza usai Satu Tahun ...,"Senin, 07 Okt 2024 10:30 WIB",Satu tahun agresi Israel ke Gaza yang dimulai ...,internasional
1,"VIDEO: Setahun Agresi Israel ke Gaza, Korban T...","Senin, 07 Okt 2024 10:03 WIB",Agresi Israel ke Jalur Gaza telah memasuki sat...,internasional
2,Jenderal Brigade Al-Quds Iran Hilang usai Isra...,"Senin, 07 Okt 2024 09:59 WIB","Kepala pasukan Brigade Al-Quds Iran, Esmail Qa...",internasional
3,VIDEO: Kelompok Militan Pakistan Serang Karach...,"Senin, 07 Okt 2024 09:28 WIB",Sebuah ledakan terjadi di dekat bandara intern...,internasional
4,"Setahun Agresi Gaza, Israel Was-was Antisipasi...","Senin, 07 Okt 2024 09:25 WIB",Pasukan Pertahanan Israel (IDF) bersiaga ketat...,internasional
...,...,...,...,...
95,VIDEO: Momen Perang Yel-yel Warnai Debat Perda...,"Minggu, 06 Okt 2024 20:42 WIB",Perang yel-yel dari tiga pendukung calon guber...,nasional
96,Dharma Sindir Gagasan RK-Pramono soal Kemaceta...,"Minggu, 06 Okt 2024 20:34 WIB","Calon gubernur jalur independen, Dharma Pongre...",nasional
97,Pramono Janji Buat Jaringan Transjakarta hingg...,"Minggu, 06 Okt 2024 20:31 WIB",Calon gubernur Jakarta Pramono Anung berjanji ...,nasional
98,Jurus RK Atasi Macet: Bikin Angkutan Sungai Hi...,"Minggu, 06 Okt 2024 20:29 WIB",Calon Gubernur Jakarta Ridwan Kamil mengungkap...,nasional


In [18]:
# Pastikan nama kolom sesuai dengan kolom yang berisi judul dan isi berita
titles = data['judul']  # Kolom judul yang sesuai
contents = data['isi']  # Kolom isi yang sesuai

# Gabungkan judul dan isi menjadi satu teks untuk setiap dokumen
data['combined_content'] = titles + " " + contents


In [26]:
import re
import pandas as pd
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from nltk.corpus import stopwords
from concurrent.futures import ThreadPoolExecutor
import nltk

# Jika stopwords belum diunduh, jalankan ini terlebih dahulu
nltk.download('stopwords')

# 2. Preprocessing Functions
def clean_lower(text):
    if isinstance(text, str):
        return text.lower()
    return text

def clean_punct(text):
    if isinstance(text, str):
        clean_patterns = re.compile(r'[0-9]|[/(){}\[\]\|@,;_]|[^a-z ]')
        text = clean_patterns.sub(' ', text)
        text = re.sub(r'\s+', ' ', text).strip()
        return text
    return text

def _normalize_whitespace(text):
    if isinstance(text, str):
        corrected = re.sub(r'\s+', ' ', text)
        return corrected.strip()
    return text

def clean_stopwords(text):
    if isinstance(text, str):
        stopword = set(stopwords.words('indonesian'))
        text = ' '.join(word for word in text.split() if word not in stopword)
        return text.strip()
    return text

# Initialize Sastrawi stemmer outside the function to avoid reinitialization
factory = StemmerFactory()
stemmer = factory.create_stemmer()

def sastrawistemmer(text):
    if isinstance(text, str):
        return ' '.join(stemmer.stem(word) for word in text.split())
    return text

# Load Data
file_path = '/content/data_100.csv'
data = pd.read_csv(file_path)

# Combine title and content into one field
data['combined_content'] = data['judul'] + " " + data['isi']

# Apply preprocessing steps
data['lower_case'] = data['combined_content'].apply(clean_lower)
data['no_punct'] = data['lower_case'].apply(clean_punct)
data['normalized'] = data['no_punct'].apply(_normalize_whitespace)
data['no_stopwords'] = data['normalized'].apply(clean_stopwords)

# Use ThreadPoolExecutor for parallelized stemming
with ThreadPoolExecutor() as executor:
    data['stemmed'] = list(executor.map(sastrawistemmer, data['no_stopwords']))

# Show a sample of the results for verification
data[['judul', 'isi', 'combined_content', 'stemmed']].head()


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,judul,isi,combined_content,stemmed
0,FOTO: Melihat Kehancuran Gaza usai Satu Tahun ...,Satu tahun agresi Israel ke Gaza yang dimulai ...,FOTO: Melihat Kehancuran Gaza usai Satu Tahun ...,foto hancur gaza agresi brutal israel agresi i...
1,"VIDEO: Setahun Agresi Israel ke Gaza, Korban T...",Agresi Israel ke Jalur Gaza telah memasuki sat...,"VIDEO: Setahun Agresi Israel ke Gaza, Korban T...",video tahun agresi israel gaza korban tewas ca...
2,Jenderal Brigade Al-Quds Iran Hilang usai Isra...,"Kepala pasukan Brigade Al-Quds Iran, Esmail Qa...",Jenderal Brigade Al-Quds Iran Hilang usai Isra...,jenderal brigade al quds iran hilang israel se...
3,VIDEO: Kelompok Militan Pakistan Serang Karach...,Sebuah ledakan terjadi di dekat bandara intern...,VIDEO: Kelompok Militan Pakistan Serang Karach...,video kelompok militan pakistan serang karachi...
4,"Setahun Agresi Gaza, Israel Was-was Antisipasi...",Pasukan Pertahanan Israel (IDF) bersiaga ketat...,"Setahun Agresi Gaza, Israel Was-was Antisipasi...",tahun agresi gaza israel was was antisipasi se...


In [27]:
tfidf_vectorizer = TfidfVectorizer()
corpus = data['stemmed'].tolist()
x_tfidf = tfidf_vectorizer.fit_transform(corpus)
df_tfidf = pd.DataFrame(x_tfidf.toarray(), columns=tfidf_vectorizer.get_feature_names_out())
df_tfidf

,abad,abai,abbas,abc,abdel,abidjan,abu,acara,acerbi,acre,...,york,young,youtube,yubileum,yudea,yudhoyono,yuri,zedong,zionis,zona
0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
1,0.133677,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
2,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.038205,0.0
3,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
4,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.057735,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.039689,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
96,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
97,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
98,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0


In [28]:
# Menerapkan SVD untuk LSA
n_components = 100  # Sesuaikan jumlah komponen sesuai kebutuhan
svd = TruncatedSVD(n_components=n_components, random_state=42)
lsa_matrix = svd.fit_transform(x_tfidf)

# Mengambil sebagian dari matriks V dan matriks Σ (Sigma)
v_matrix = svd.components_  # Matriks V (komponen term)
singular_values = svd.singular_values_  # Singular values (Sigma)

# Menentukan berapa banyak komponen dari V dan Sigma yang ingin digunakan
partial_n = 50  # Misalkan kita hanya ingin mengambil 50 komponen teratas

# Ambil sebagian dari matriks V dan Sigma
partial_v_matrix = v_matrix[:partial_n]  # Ambil 50 baris pertama dari V
partial_sigma = np.diag(singular_values[:partial_n])  # Buat matriks diagonal dengan 50 singular values terbesar

In [30]:
data_svd = pd.DataFrame(lsa_matrix)

In [31]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Load data
data = pd.read_csv('/content/data_100.csv')

# Extract titles and contents
titles = data['judul']  # Column containing titles
contents = data['isi']  # Column containing contents

# Combine titles and contents into one text for each document
documents = titles + " " + contents

# TF-IDF on the combined text
vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = vectorizer.fit_transform(documents)

# Apply SVD for LSA
n_components = 100
svd = TruncatedSVD(n_components=n_components, random_state=42)
lsa_matrix = svd.fit_transform(tfidf_matrix)

# Extract parts of matrix V and singular values (Sigma)
v_matrix = svd.components_
singular_values = svd.singular_values_
partial_n = 50
partial_v_matrix = v_matrix[:partial_n]
partial_sigma = np.diag(singular_values[:partial_n])

# Search function
def search_documents(query, top_n=5):
    query_tfidf = vectorizer.transform([query])
    query_lsa = svd.transform(query_tfidf)
    similarities = cosine_similarity(lsa_matrix[:, :partial_n], query_lsa[:, :partial_n]).flatten()
    top_indices = similarities.argsort()[-top_n:][::-1]
    results = pd.DataFrame({
        'Judul': titles.iloc[top_indices].values,
        'Isi': contents.iloc[top_indices].values,
        'Skor Kemiripan': similarities[top_indices]
    })
    return results

# Example usage
query = "sepak bola"
results = search_documents(query)
results


,Judul,Isi,Skor Kemiripan
0,"Pramono: 354 Ribu Orang Menganggur di Jakarta,...",Calon Gubernur DKI Jakarta Pramono Anung menga...,0.0
1,Kongo Mulai Gelar Vaksinasi Wabah Mpox,Republik Demokratik Kongo meluncurkan kampanye...,0.0
2,WNI yang Dievakuasi dari Lebanon Akan Terbang ...,Sebanyak 40 warga negara Indonesia (WNI) dan s...,0.0
3,RUDAL: 7 Julukan Indonesia yang Mendunia,Indonesia merupakan salah satu negara yang akt...,0.0
4,"3 Negara Barat Tembak Rudal Iran ke Israel, Bi...","Pemimpin Tertinggi Iran, Ayatollah Ali Khamene...",0.0
